In [5]:
#предобработка текстов
import re
import numpy as np
import nltk
nltk.download('stopwords')
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2
#from gensim.test.utils import common_texts
import pandas as pd
from gensim.corpora.dictionary import Dictionary

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [10]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [11]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [6]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-8-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 25.4 s


In [16]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 6min 27s


In [17]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [18]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [19]:
common_dictionary[10]

'ватутин'

In [20]:
from gensim.models import LdaModel

In [21]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min 46s


In [22]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [23]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(7, 0.97833556)]

In [24]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: nn исследование гражданин ребёнок который день это
topic_1: научный университет год миссия планета восток китай
topic_2: доклад смерть бензин звонок зампред публикация скрываться
topic_3: суд млн убийство следователь обвинение подозревать задержание
topic_4: лечение квартира парламент кость акционер пищевой партия
topic_5: взрыв это который медведев изучение завод страна
topic_6: станция градус египет одежда температура сон употребление
topic_7: ракета запуск это первый поверхность пациент который
topic_8: обнаружить район произойти город пострадать данные человек
topic_9: год это весь рубль млрд который тыс
topic_10: слияние корейский часовой возведение планета жюри южнокорейский
topic_11: год российский это россия nn который страна
topic_12: городской белый греция грузия преступник болгария дом
topic_13: это который мочь год компания россия свой
topic_14: земля проверка топливо северный памятник температура значительно
topic_15: который человек это год свой население nn
topi

In [25]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [26]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.040254,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.036709,0.0,0.0,0.0,0.0,0.0,0.000000
1,4896,0.0,0.000000,0.0,0.051251,0.0,0.687769,0.000000,0.069033,0.0,...,0.0,0.0,0.0,0.170938,0.0,0.0,0.0,0.0,0.0,0.000000
2,4897,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.978336,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,4898,0.0,0.000000,0.0,0.000000,0.0,0.020559,0.013262,0.501429,0.0,...,0.0,0.0,0.0,0.035273,0.0,0.0,0.0,0.0,0.0,0.000000
4,4899,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.053678,0.000000,0.0,...,0.0,0.0,0.0,0.164706,0.0,0.0,0.0,0.0,0.0,0.338488


In [27]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [28]:
doc_dict[293622]

array([0.25145483, 0.        , 0.        , 0.        , 0.        ,
       0.12521629, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.50960302, 0.        , 0.        , 0.        , 0.03685495,
       0.        , 0.06473602, 0.        , 0.        , 0.        ])

In [29]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [30]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.212173,0.000000,0.0,0.005120,0.000000,0.067642,0.001758,0.004233,0.049783,...,0.238127,0.004857,0.010952,0.000000,0.085100,0.018972,0.010789,0.008565,0.000000,0.005566
1,u108690,0.050766,0.005602,0.0,0.009654,0.002475,0.018178,0.000000,0.022140,0.047035,...,0.135527,0.000000,0.009599,0.003771,0.102232,0.015063,0.024845,0.000000,0.002458,0.042431
2,u108339,0.076091,0.005925,0.0,0.012994,0.004545,0.007070,0.001781,0.008793,0.099169,...,0.156279,0.000000,0.008936,0.000000,0.114550,0.009984,0.000000,0.019764,0.000000,0.032944


In [31]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [32]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.212173,0.000000,0.0,0.005120,0.000000,0.067642,0.001758,0.004233,0.049783,...,0.004857,0.010952,0.000000,0.085100,0.018972,0.010789,0.008565,0.000000,0.005566,0
1,u108690,0.050766,0.005602,0.0,0.009654,0.002475,0.018178,0.000000,0.022140,0.047035,...,0.000000,0.009599,0.003771,0.102232,0.015063,0.024845,0.000000,0.002458,0.042431,1
2,u108339,0.076091,0.005925,0.0,0.012994,0.004545,0.007070,0.001781,0.008793,0.099169,...,0.000000,0.008936,0.000000,0.114550,0.009984,0.000000,0.019764,0.000000,0.032944,1


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [34]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [35]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [36]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.0556035 , 0.23019937, 0.27899112, 0.16281223, 0.01137002,
       0.01138826, 0.17148449, 0.04683386, 0.37215756, 0.02600399])

In [37]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [38]:
import itertools

In [39]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.248800, F-Score=0.671, Precision=0.572, Recall=0.812


In [40]:
roc_auc_score(y_test, preds)

0.9437223094365952

## Начало дз

### median

In [41]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

In [42]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.233816,0.0,0.0,0.0,0.0,0.051272,0.0,0.0,0.027305,...,0.264316,0.0,0.000000,0.0,0.046006,0.0,0.00000,0.0,0.0,0.000000
1,u108690,0.044051,0.0,0.0,0.0,0.0,0.014834,0.0,0.0,0.022589,...,0.129589,0.0,0.000000,0.0,0.050315,0.0,0.01408,0.0,0.0,0.008677
2,u108339,0.079941,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.091711,...,0.147280,0.0,0.007318,0.0,0.117997,0.0,0.00000,0.0,0.0,0.020604


In [43]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [44]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.233816,0.0,0.0,0.0,0.0,0.051272,0.0,0.0,0.027305,...,0.0,0.000000,0.0,0.046006,0.0,0.00000,0.0,0.0,0.000000,0
1,u108690,0.044051,0.0,0.0,0.0,0.0,0.014834,0.0,0.0,0.022589,...,0.0,0.000000,0.0,0.050315,0.0,0.01408,0.0,0.0,0.008677,1
2,u108339,0.079941,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.091711,...,0.0,0.007318,0.0,0.117997,0.0,0.00000,0.0,0.0,0.020604,1


In [45]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [46]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [47]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.1105496 , 0.11423731, 0.40203282, 0.3073678 , 0.00664427,
       0.02105211, 0.07299779, 0.01364671, 0.43613464, 0.03667816])

In [48]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.317170, F-Score=0.772, Precision=0.727, Recall=0.824


In [49]:
roc_auc_score(y_test, preds)

0.97136112564684

### max

In [51]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [52]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.465127,0.000000,0.0,0.030721,0.000000,0.178090,0.010546,0.025397,0.187590,...,0.509603,0.029145,0.065714,0.000000,0.305072,0.061826,0.064736,0.051392,0.000000,0.033398
1,u108690,0.124801,0.023551,0.0,0.029037,0.014849,0.051227,0.000000,0.104682,0.153456,...,0.223102,0.000000,0.057596,0.022624,0.374120,0.059001,0.070604,0.000000,0.014745,0.200503
2,u108339,0.156753,0.021914,0.0,0.044780,0.027269,0.030344,0.010689,0.030549,0.197359,...,0.269424,0.000000,0.020217,0.000000,0.214750,0.059903,0.000000,0.090946,0.000000,0.106731


In [53]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [54]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.465127,0.000000,0.0,0.030721,0.000000,0.178090,0.010546,0.025397,0.187590,...,0.029145,0.065714,0.000000,0.305072,0.061826,0.064736,0.051392,0.000000,0.033398,0
1,u108690,0.124801,0.023551,0.0,0.029037,0.014849,0.051227,0.000000,0.104682,0.153456,...,0.000000,0.057596,0.022624,0.374120,0.059001,0.070604,0.000000,0.014745,0.200503,1
2,u108339,0.156753,0.021914,0.0,0.044780,0.027269,0.030344,0.010689,0.030549,0.197359,...,0.000000,0.020217,0.000000,0.214750,0.059903,0.000000,0.090946,0.000000,0.106731,1


In [55]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [56]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [57]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.01054819, 0.00589304, 0.41212703, 0.30915747, 0.10561279,
       0.0070677 , 0.0103928 , 0.04600334, 0.27725993, 0.05345885])

In [58]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.316479, F-Score=0.739, Precision=0.702, Recall=0.780


In [59]:
roc_auc_score(y_test, preds)

0.9589301703587417

### Вывод

### mean
est Threshold=0.278081, F-Score=0.655, Precision=0.641, Recall=0.669 / 0.9471527414384557
### median
Best Threshold=0.317170, F-Score=0.772, Precision=0.727, Recall=0.824 / 0.97136112564684
### max
Best Threshold=0.316479, F-Score=0.739, Precision=0.702, Recall=0.780 / 0.9589301703587417